In [1]:
import pretty_midi
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
# pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:

def midi_to_df(location):

    midi_data = pretty_midi.PrettyMIDI(location)
    data = []

    # Calculate the maximum time and total seconds
    time = max(note.end for instrument in midi_data.instruments for note in instrument.notes)
    intervals = round(time / 60)
    total_seconds = intervals * 60

    # Loop through instruments and notes
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            # Ensure note's end time does not exceed total_seconds
            start_time = note.start
            end_time = min(note.end, total_seconds)
            pitch = note.pitch

            # Only print notes within the total_seconds limit
            if start_time <= total_seconds:
                start_segment = int(start_time / 60)
                end_segment = int(end_time / 60)
                if end_time-(60*start_segment) >= 60:
                    next_end = end_time-(60*(start_segment+1))
                    next_start = 0.00
                    end_time = 59.99


                    data.append({
                        "minute": start_segment,
                        "start_time": round(start_time-(60*start_segment), 3),
                        "end_time": round(end_time, 3),
                        "pitch": pitch,
                    })

                    if intervals != start_segment +1:

                        data.append({
                            "minute": start_segment + 1,
                            "start_time": round(next_start, 3),
                            "end_time": round(next_end, 3),
                            "pitch": pitch,
                        })

                else:
                    end_time = end_time-(60*start_segment)


                    data.append({
                        "minute": start_segment,
                        "start_time": round(start_time-(60*start_segment), 3),
                        "end_time": round(end_time, 3),
                        "pitch": pitch,
                    })

    df = pd.DataFrame(data)
    df = df.sort_values(by=["minute", "start_time"], ascending=True).reset_index(drop=True)
    return df


In [12]:
df = midi_to_df("data/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi")
display(df.head())
display(df.tail())

,minute,start_time,end_time,pitch
0,0,1.093,1.190,71
1,0,1.279,1.497,55
2,0,1.289,1.794,71
3,0,1.464,1.631,59
4,0,1.633,1.753,62


,minute,start_time,end_time,pitch
7838,15,59.659,59.990,72
7839,15,59.797,59.955,42
7840,15,59.801,59.901,62
7841,15,59.936,59.990,45
7842,15,59.945,59.990,66


In [5]:
# Determine the maximum minute value
max_minute = df['minute'].max()

# Define time increment (approximately 11 milliseconds)
time_increment = 0.0116099071207  # In seconds

# Generate time intervals for each minute
time_dfs = []
for minute in range(max_minute + 1):
    # Create 'seconds' from 0 up to 60 seconds (not including 60)
    seconds = np.arange(0, 60, time_increment)
    # Ensure 'seconds' does not exceed 59.99 seconds
    seconds = seconds[seconds < 59.99]
    # Create a DataFrame for this minute
    time_df = pd.DataFrame({
        'minute': minute,
        'seconds': seconds
    })
    time_dfs.append(time_df)

# Combine all time DataFrames
time_df = pd.concat(time_dfs, ignore_index=True)

# Reset index to ensure continuous indexing
time_df.reset_index(drop=True, inplace=True)

# Initialize the Output Matrix
num_times = len(time_df)
num_pitches = 88  # 88 piano keys (MIDI notes 21 to 108)
piano_roll = np.zeros((num_times, num_pitches), dtype=int)

# Iterate over each note in the MIDI dataframe
for index, row in df.iterrows():
    minute = row['minute']
    start_seconds = row['start_time']
    end_seconds = row['end_time']
    pitch = row['pitch']
    pitch_index = int(pitch) - 21  # MIDI note 21 corresponds to index 0

    if 0 <= pitch_index < num_pitches:
        # Find indices where the note is active
        # Create masks for minute and seconds
        minute_mask = time_df['minute'] == minute
        seconds_mask = (time_df['seconds'] >= start_seconds) & (time_df['seconds'] < end_seconds)
        # Combine masks
        active_indices = time_df[minute_mask & seconds_mask].index

        # Set the corresponding entries to 1
        piano_roll[active_indices, pitch_index] = 1
    else:
        print(f"Pitch {pitch} is out of piano range.")

# Create column names for each pitch
pitch_columns = [f'pitch_{pitch}' for pitch in range(21, 109)]  # MIDI notes 21 to 108

# Create a DataFrame from the piano_roll matrix
piano_roll_df = pd.DataFrame(piano_roll, columns=pitch_columns)

# Insert the 'minute' and 'seconds' columns
piano_roll_df.insert(0, 'seconds', time_df['seconds'])
piano_roll_df.insert(0, 'minute', time_df['minute'])

piano_roll_df.head(20)

,minute,seconds,pitch_21,pitch_22,pitch_23,pitch_24,pitch_25,pitch_26,pitch_27,pitch_28,pitch_29,pitch_30,pitch_31,pitch_32,pitch_33,pitch_34,pitch_35,pitch_36,pitch_37,pitch_38,pitch_39,pitch_40,pitch_41,pitch_42,pitch_43,pitch_44,pitch_45,pitch_46,pitch_47,pitch_48,pitch_49,pitch_50,pitch_51,pitch_52,pitch_53,pitch_54,pitch_55,pitch_56,pitch_57,pitch_58,pitch_59,pitch_60,pitch_61,pitch_62,pitch_63,pitch_64,pitch_65,pitch_66,pitch_67,pitch_68,pitch_69,pitch_70,pitch_71,pitch_72,pitch_73,pitch_74,pitch_75,pitch_76,pitch_77,pitch_78,pitch_79,pitch_80,pitch_81,pitch_82,pitch_83,pitch_84,pitch_85,pitch_86,pitch_87,pitch_88,pitch_89,pitch_90,pitch_91,pitch_92,pitch_93,pitch_94,pitch_95,pitch_96,pitch_97,pitch_98,pitch_99,pitch_100,pitch_101,pitch_102,pitch_103,pitch_104,pitch_105,pitch_106,pitch_107,pitch_108
0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0.011610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0.023220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0.034830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0.046440,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0.058050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0.069659,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0.081269,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0.092879,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0.104489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
piano_roll_df.iloc[5160:5175]

,minute,seconds,pitch_21,pitch_22,pitch_23,pitch_24,pitch_25,pitch_26,pitch_27,pitch_28,pitch_29,pitch_30,pitch_31,pitch_32,pitch_33,pitch_34,pitch_35,pitch_36,pitch_37,pitch_38,pitch_39,pitch_40,pitch_41,pitch_42,pitch_43,pitch_44,pitch_45,pitch_46,pitch_47,pitch_48,pitch_49,pitch_50,pitch_51,pitch_52,pitch_53,pitch_54,pitch_55,pitch_56,pitch_57,pitch_58,pitch_59,pitch_60,pitch_61,pitch_62,pitch_63,pitch_64,pitch_65,pitch_66,pitch_67,pitch_68,pitch_69,pitch_70,pitch_71,pitch_72,pitch_73,pitch_74,pitch_75,pitch_76,pitch_77,pitch_78,pitch_79,pitch_80,pitch_81,pitch_82,pitch_83,pitch_84,pitch_85,pitch_86,pitch_87,pitch_88,pitch_89,pitch_90,pitch_91,pitch_92,pitch_93,pitch_94,pitch_95,pitch_96,pitch_97,pitch_98,pitch_99,pitch_100,pitch_101,pitch_102,pitch_103,pitch_104,pitch_105,pitch_106,pitch_107,pitch_108
5160,0,59.907121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5161,0,59.918731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5162,0,59.930341,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5163,0,59.941950,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5164,0,59.953560,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5165,0,59.965170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5166,0,59.976780,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5167,0,59.988390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5168,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5169,1,0.011610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
piano_roll_df.shape

(82688, 90)

In [8]:
piano_roll_df.tail()

,minute,seconds,pitch_21,pitch_22,pitch_23,pitch_24,pitch_25,pitch_26,pitch_27,pitch_28,pitch_29,pitch_30,pitch_31,pitch_32,pitch_33,pitch_34,pitch_35,pitch_36,pitch_37,pitch_38,pitch_39,pitch_40,pitch_41,pitch_42,pitch_43,pitch_44,pitch_45,pitch_46,pitch_47,pitch_48,pitch_49,pitch_50,pitch_51,pitch_52,pitch_53,pitch_54,pitch_55,pitch_56,pitch_57,pitch_58,pitch_59,pitch_60,pitch_61,pitch_62,pitch_63,pitch_64,pitch_65,pitch_66,pitch_67,pitch_68,pitch_69,pitch_70,pitch_71,pitch_72,pitch_73,pitch_74,pitch_75,pitch_76,pitch_77,pitch_78,pitch_79,pitch_80,pitch_81,pitch_82,pitch_83,pitch_84,pitch_85,pitch_86,pitch_87,pitch_88,pitch_89,pitch_90,pitch_91,pitch_92,pitch_93,pitch_94,pitch_95,pitch_96,pitch_97,pitch_98,pitch_99,pitch_100,pitch_101,pitch_102,pitch_103,pitch_104,pitch_105,pitch_106,pitch_107,pitch_108
82683,15,59.94195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
82684,15,59.95356,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
82685,15,59.96517,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
82686,15,59.97678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
82687,15,59.98839,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
